In [2]:
import pandas as pd
import pickle
import os
import numpy as np
import random
from colog.colog import colog
from datetime import datetime, timedelta
c = colog()

In [35]:
def load_data_one_stock(path):
  load_data = pd.DataFrame()
  try:
    file = open(path, 'rb')
    load_data= pickle.load(file)
    file.close()
  except Exception as e:
    print(e)
  return load_data

In [37]:
# SETTINGS
# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA']
# stock_name_list  = ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']

# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA'] + ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']
stock_name_list = []
stock_name_list += ['GOOG','JPM','XOM','UNH','JNJ','V','AVGO','PG','LLY','MA','HD','CVX','MRK', 
                       'PEP','COST','ABBV','ADBE','KO','CRM','WMT','MCD','CSCO','BAC','PFE','TMO','ACN','NFLX','ABT','AMD','LIN','ORCL','CMCSA',
                       'TXN','DIS','WFC','DHR','PM','NEE','VZ','INTC','RTX','HON','LOW','UPS','INTU','SPGI','NKE','COP','QCOM','BMY','CAT','UNP','BA','ISRG',
                        'GE','IBM','AMGN','AMAT','MDT','SBUX','PLD','NOW','MS','DE','BLK','GS','T','LMT','AXP','SYK','ADI','TJX','ELV','MDLZ','GILD','ADP','MMC',
                        'C','AMT','CVS','VRTX','SCHW','LRCX','MO','TMUS','SLB', 'ETN', 'ZTS', 'CI', 'PYPL']

stock_name_list += ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE',
                       'CL','SHW','WM','HCA','TGT','FCX','FDX','F','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
                       'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
                       'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
                       'BIIB','SPG','MSCI','DHI','ROK']

folder_saved_models = f'historical_data\\period2yinterval1h'
parent_path = os.getcwd()
buy_money = 1000
commission_value = 0.001
trainig_part = 0.5
test_part = 0.5

Exlude companies with bad ration and retraining!!!

Currten best settings:
{'is_near_global_max_prt': 0, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.2, 'buy_ratio_border': 10}
-3458.0764813327787 495


In [5]:
## Load data

# file_name = 'df_stock_AAPL_period2y_interval1h.pkl'
# path = os.path.join(parent_path, folder_saved_models, file_name)
## stock_name = item[str.find(item, 'df_') + 3 : str.find(item, '_20')]
# df = load_data_one_stock(path)
## df.plot()
## print(df)

In [6]:
def is_near_global_max(i, k=400, prt=70):

  if i > k:
   gmax = max(df['close'].iloc[i - k: i])
   reference_point = df['close'].iloc[i - k]
   gmin = min(df['close'].iloc[i - k: i])
  else:
   reference_point = df['close'].iloc[0]
   gmax = max(df['close'].iloc[0: i])
   gmin = min(df['close'].iloc[0: i])
  # if gmax == reference_point:
  #   result = False
  # elif 100 * (df['close'].iloc[i] - reference_point) / (gmax - reference_point) > prt:
  #   result = True
  # else:
  #   result = False
  result = 100 * (df['close'].iloc[i] - gmin) / (gmax - gmin) > prt

  return result

# def is_near_global_max(i, k=400, prt=70):

#   if i > k:
#    gmax = max(df['close'].iloc[i - k: i])
#    reference_point = df['close'].iloc[i - k]
#    gmin = min(df['close'].iloc[i - k: i])
#   else:
#    reference_point = df['close'].iloc[0]
#    gmax = max(df['close'].iloc[0: i])
#    gmin = min(df['close'].iloc[0: i])

#   if gmax == reference_point:
#     result = False
#   elif 100 * (df['close'].iloc[i] - reference_point) / (gmax - reference_point) > prt:
#     result = True
#   else:
#     result = False

#   return result

def global_max_prop(i, k=40):

  if i > k:
   gmax = max(df['close'].iloc[i - k: i])
   gmin = min(df['close'].iloc[i - k: i])
  else:
   gmax = max(df['close'].iloc[0: i])
   gmin = min(df['close'].iloc[0: i])

  return gmax / gmin

def number_red_candles(df, i, k=11):

  if i < k:
    number_red_candles = (df['ha_colour'][0 : i] == 'red').sum()
  else:
    number_red_candles = (df['ha_colour'][i - k : i] == 'red').sum()
  return number_red_candles

def number_green_candles(df, i, k=11):

  if i < k:
    number_green_candles = (df['ha_colour'][0 : i] == 'green').sum()
  else:
    number_green_candles = (df['ha_colour'][i - k : i] == 'green').sum()
  return number_green_candles

In [41]:
def one_stock_simulation(df, ticker, mode, _gain_coef = 1.005, _lose_coef = 0.97, print_sim_results=False, print_trans_history=False):
  '''
    Parameters:
      mode: train / full / test

    Returns:
      profits, wins, loses 
  '''

  history_log = {'number success': 0,
                'number fails': 0,
                'number wins': 0,
                'proportion': 0,
                'profit': 0,
                'transactions': [] }  
  last_top = df['close'].iloc[0]
  last_top_time = df['close'].index[0]
  last_top_i = 0
  reverse_flag = False
  sell_finished = True
  buy_order_placed = False
  buy_price = 0
  
  if mode == 'train':
    range_ = range(2, int(df.shape[0] * trainig_part))
  if mode == 'full': 
    range_ = range(2, df.shape[0] - 1)
  if mode == 'test':
    range_ = range(int(df.shape[0] * trainig_part) + 1, df.shape[0] - 1)

  last_top_dict = {'time': [], 'value': []}  

  for i in range_:

    # dynaminc profit/loose coefficient
    if i > 200:
      if df['close'].iloc[i] / df['close'].iloc[i - 200] > bull_trend_coef: # bull trend
        gain_coef = 1.005
        lose_coef = 0.97  # not equal to 1 - k (DO Different Expertiments)
      else: # bear trend
        gain_coef = _gain_coef
        lose_coef = _lose_coef  # used to be 0.95 16/08/2024
        # print(f'lose coef {lose_coef}')
    else: # undefined trend
      gain_coef = 1.005
      lose_coef = 0.97

    if sell_finished == False and df['high'].iloc[i] / buy_price > gain_coef:
      # sell_price = float(df['high'].iloc[i])
      sell_price = buy_price * gain_coef
      history_log['number success'] += 1
      number_buy_stocks = int(buy_money / buy_price)
      history_log['profit'] += (sell_price - buy_price) * number_buy_stocks - commission_value * (sell_price + buy_price) * number_buy_stocks
      history_log['number wins'] += 1
      history_log['transactions'].append({
        'type': 'success',
        'buy_price' : round(buy_price, 2),
        'sell_price': round(sell_price, 2), 
        'ratio': round(buy_ratio_hl ,2),
        'buy_time' : buy_time,
        'sell_time' : df['high'].index[i],
        'buy_lose_coef': buy_lose_coef,
        'buy_gain_coef': buy_gain_coef,
     })
      sell_finished = True
      buy_order_placed = False

    if sell_finished == False and df['low'].iloc[i] / buy_price < lose_coef :
      # sell_price = float(df['low'].iloc[i])
      sell_price = buy_price * lose_coef
      sell_finished = True
      buy_order_placed = True
      number_buy_stocks = int(buy_money / buy_price)
      history_log['profit'] += (sell_price - buy_price) * number_buy_stocks - commission_value * (sell_price + buy_price) * number_buy_stocks
      history_log['number fails'] += 1
      history_log['transactions'].append({
        'type': 'fail',
        'buy_price' : round(buy_price, 2),
        'sell_price': round(sell_price, 2),
        'ratio': round(buy_ratio_hl ,2),
        'buy_time' : buy_time,
        'sell_time' : df['low'].index[i],
        'buy_lose_coef': buy_lose_coef,
        'buy_gain_coef': buy_gain_coef,
      })

    # KEY parameters: RIV, prt, last_top_ratio
    # buy_time_cond = (df.index[i]).hour <= 15 and (df.index[i]).hour >= 11
    # last_top_df = pd.DataFrame(last_top_dict)
    # if last_top_df.shape[0] > 1:
    #   last_top_df = last_top_df.loc[last_top_df['time'] > (df.index[i] - pd.Timedelta(hours=24))]
    #   last_top_24hours = float(last_top_df['value'].max())
    # else:
    #   last_top_24hours = df['high'].iloc[i]
    # last_top_24hours_proportion = float(last_top_24hours -  df['open'].iloc[i]) / float(df['ha_pct'].iloc[i])

    # delta_hours = 48
    # if df.shape[0] > delta_hours -1:
    #   max = df['close'].iloc[i - delta_hours: i].max()
    #   min = df['close'].iloc[i - delta_hours: i].min()

    #   last_top_24hours_proportion = (df['open'].iloc[i] - min) / (max - min)
    buy_time_cond = True

    buy_ratio = float(last_top -  df['open'].iloc[i]) / float(df['ha_pct'].iloc[i])
    if df['ha_pct'].iloc[i] > RIV \
      and sell_finished == True \
      and buy_time_cond \
      and number_red_candles(df,i) > 6  \
      and last_top / df['open'].iloc[i] > last_top_ratio  \
      and i - last_top_i > distance_from_last_top  \
      and not(is_near_global_max(i, k=400, prt=is_near_global_max_prt)) \
      and buy_ratio > buy_ratio_border:
        # and reverse_flag==False \
         
      # and global_max_prop(i, k=40) < global_max_prop_ratio \
      # and not(is_near_global_max(i, k = 40, prt = 101)):
   
      reverse_flag = True
      # buy_price = float(df['close'].iloc[i])
      # buy_price = float(df['open'].iloc[i + 1])
      # buy_price = min(df['close'].iloc[i - 1], df['open'].iloc[i - 1], df['close'].iloc[i], df['open'].iloc[i] )
      # if number_red_candles(df, i, k=2) == 0 and number_green_candles(df, i, k=10) < 5 \
      #   or number_green_candles(df, i, k=10) > 7:
      #   # buy_price = (df['close'].iloc[i] + df['open'].iloc[i]) / 2
      #   buy_price = df['close'].iloc[i]
      # else:
      # buy_price = (df['close'].iloc[i] + df['open'].iloc[i]) / 2
      buy_price = min(df['close'].iloc[i - 10: i].min(), df['open'].iloc[i - 10: i].min() )
      buy_time = df.index[i]
      buy_ratio_hl = buy_ratio
      buy_gain_coef = gain_coef
      buy_lose_coef = lose_coef
      # sell_finished = False
      buy_order_placed = True
      if df['low'].iloc[i + 1] < buy_price and buy_order_placed:
        # buy_order_placed = True
        sell_finished = False
    
    if df['ha_pct'].iloc[i] > RIV \
      and False \
      and sell_finished == True \
      and buy_time_cond \
      and reverse_flag==False \
      and number_red_candles(df, i, k=3) < 2 \
      and number_green_candles(df, i, k=4) >= 3 \
      and number_green_candles(df, i, k=15) < 10 \
      and number_red_candles(df, i, k=3) != 3:
      
      # and number_red_candles(df, i, k=3) != 3 \
      # and number_green_candles(df, i, k=15) < 10 \
  
      
      reverse_flag = True
      # buy_price = float(df['close'].iloc[i])
      # buy_price = float(df['open'].iloc[i + 1])
      # buy_price = min(df['close'].iloc[i - 1], df['open'].iloc[i - 1], df['close'].iloc[i], df['open'].iloc[i] 
      buy_price = (df['close'].iloc[i - 1] + df['open'].iloc[i - 1]) / 2
      # buy_price = df['close'].iloc[i]
      # buy_price = min(df['close'].iloc[i - 1], df['open'].iloc[i - 1], df['close'].iloc[i], df['open'].iloc[i] )
     
      buy_time = df.index[i]
      buy_ratio_hl = buy_ratio
      buy_gain_coef = gain_coef
      buy_lose_coef = lose_coef
      # sell_finished = False
      buy_order_placed = True
      if df['low'].iloc[i] < buy_price and buy_order_placed:
        buy_order_placed = True
        sell_finished = False
        
    # last top and reverse flag
    if df['ha_colour'].iloc[i] == 'red' \
      and df['ha_colour'].iloc[i - 1] == 'green'\
      and df['ha_colour'].iloc[i - 2] == 'green'\
      and df['ha_pct'].iloc[i - 1] > 0.1 \
      and df['ha_pct'].iloc[i - 2] > 0.1:
  
      last_top = df['high'].iloc[i - 1]
      last_top_i = i - 1
      reverse_flag = False
      # last_top_dict['time'].append(df.index[i])
      # last_top_dict['value'].append(last_top)
  
  history_log['proportion'] =   history_log['number wins'] / ( history_log['number fails'] + 1)

  # Simulation results 

  if print_sim_results:
    print('-'*50)
    print(f'Simulation results for ticker {ticker}:')
    print(history_log)

    df_tran = pd.DataFrame(history_log['transactions'])
    print('Statictic for buy gain coef = 1.02')
    if df_tran.shape[0] > 0:
      print(df_tran.loc[df_tran['buy_gain_coef'] == 1.02]['type'].value_counts())
      print(f'Statictic for buy_gain coef = {_gain_coef}')
      print(df_tran.loc[df_tran['buy_gain_coef'] == _gain_coef]['type'].value_counts())
    
  if print_trans_history:
    for tran in history_log['transactions']:
      if tran['buy_gain_coef'] == 1.02:
        c.print(tran, color='green')
      else:
        c.print(tran, color='blue')

  return history_log['profit'], history_log['number wins'], history_log['number fails']


In [493]:
sum = 50000
for i in range(100):
  day_profit = sum * 0.005
  sum = sum * 1.005
  print(f'day {i}, sum is {sum:.2f}, day profit is {day_profit:.2f}')

day 0, sum is 50250.00, day profit is 250.00
day 1, sum is 50501.25, day profit is 251.25
day 2, sum is 50753.76, day profit is 252.51
day 3, sum is 51007.53, day profit is 253.77
day 4, sum is 51262.56, day profit is 255.04
day 5, sum is 51518.88, day profit is 256.31
day 6, sum is 51776.47, day profit is 257.59
day 7, sum is 52035.35, day profit is 258.88
day 8, sum is 52295.53, day profit is 260.18
day 9, sum is 52557.01, day profit is 261.48
day 10, sum is 52819.79, day profit is 262.79
day 11, sum is 53083.89, day profit is 264.10
day 12, sum is 53349.31, day profit is 265.42
day 13, sum is 53616.06, day profit is 266.75
day 14, sum is 53884.14, day profit is 268.08
day 15, sum is 54153.56, day profit is 269.42
day 16, sum is 54424.33, day profit is 270.77
day 17, sum is 54696.45, day profit is 272.12
day 18, sum is 54969.93, day profit is 273.48
day 19, sum is 55244.78, day profit is 274.85
day 20, sum is 55521.00, day profit is 276.22
day 21, sum is 55798.61, day profit is 277.6

In [13]:
buy_time_cond = True
buy_ratio_border = 7
distance_from_last_top  = 0  # NOT SIGNIFICANT?
last_top_ratio = 1.002 # last top / current open price  NOT SIGNIFICANT?
RIV = 0.1 # Reversal indication Value
is_near_global_max_prt = 96 # usd to be 96
bull_trend_coef = 1.12
last_top_24hours_proportion_border = 1

# best_params = {'is_near_global_max_prt': 80, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.25, 'buy_ratio_border': 9, 'bull_trend_coef': 1.12}
# best_params = {'is_near_global_max_prt': 96, 'distance_from_last_top': 0, 'last_top_ratio': 1.001, 'RIV': 0.15, 'buy_ratio_border': 5, 'bull_trend_coef': 1.12}
# best_params = {'is_near_global_max_prt': 96, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
# after last check
best_params = {'is_near_global_max_prt': 120, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.05,
                'buy_ratio_border': 0, 'bull_trend_coef': 1.07, 'last_top_24hours_proportion_border': 1}

for param in best_params:
  vars()[param] = best_params[param]

parameters_list_search = {
  'is_near_global_max_prt' : [0, 20, 30, 50, 70, 80, 96, 100],
  'distance_from_last_top': [0,2,3,5],
  'last_top_ratio': [1, 1.005, 1.01, 1.05, 1.1],
  'RIV' : [0.05, 0.1, 0.15, 0.2, 0.25],
  'buy_ratio_border' : [0,3,5,7,9,10],
  'bull_trend_coef': [1.07, 1.12]
}

parameters_list_search = {
  # 'is_near_global_max_prt2' : np.arange(80,130,10)
  'RIV' : [0.05, 0.1, 0.15, 0.2, 0.25]
  # 'global_max_prop_ratio': np.arange(1.02,1.2,0.02)
  # 'bull_trend_coef': [1, 1.05, 1.07, 1.12]
}

# {'is_near_global_max_prt': 70, 'distance_from_last_top': 0, 'last_top_ratio': 1.1, 'RIV': 0.2, 'buy_ratio_border': 7, 'bull_trend_coef': 1.07}

# parameters_list_search = {
#   # 'last_top_24hours_proportion_border': np.arange(1, 0, -0.1)
#   'RIV' : [0.05, 0.1, 0.15, 0.2, 0.25]
# }

In [268]:
stock_name_list = []
stock_name_list += ['GOOG','JPM','XOM','UNH','JNJ','V','AVGO','PG','LLY','MA','HD','CVX','MRK', 
                       'PEP','COST','ABBV','ADBE','KO','CRM','WMT','MCD','CSCO','BAC','PFE','TMO','ACN','NFLX','ABT','AMD','LIN','ORCL','CMCSA',
                       'TXN','DIS','WFC','DHR','PM','NEE','VZ','INTC','RTX','HON','LOW','UPS','INTU','SPGI','NKE','COP','QCOM','BMY','CAT','UNP','BA','ISRG',
                        'GE','IBM','AMGN','AMAT','MDT','SBUX','PLD','NOW','MS','DE','BLK','GS','T','LMT','AXP','BKNG','SYK','ADI','TJX','ELV','MDLZ','GILD','ADP','MMC',
                        'C','AMT','CVS','VRTX','SCHW','LRCX','MO','TMUS','SLB', 'ETN', 'ZTS', 'CI', 'PYPL']

stock_name_list += ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE','ATVI',
                       'CL','SHW','WM','HCA','TGT','FCX','FDX','F','ORLY','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
                       'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
                       'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
                       'BIIB','SPG','MSCI','DHI','ROK']
len(stock_name_list)

191

In [15]:
# BEST PARAMETERS OPTIMIZATION  and OPTIMAL STOCK LIST
best_profit = -10**5
best_proportion = 0
best_profit_improved = True
optimize_profit = True
optimize_proportion = False
ticker_stats = {}
stock_name_list_opt = stock_name_list[:]
# for parameter in parameters_list_search:
while best_profit_improved:
  for i in random.sample(range(len(parameters_list_search)), len(parameters_list_search)):
    parameter = list(parameters_list_search.keys())[i]
    total_profit_list = []
    total_loses_list = []
    total_proportion_list = []
    for value in parameters_list_search[parameter]:
      vars()[parameter] = value
      c.print(f'parametr is {parameter} value is {value}:', color='white')
      total_profit = 0
      total_loses = 0
      total_wins = 0
      for ticker in stock_name_list_opt:
        file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
        path = os.path.join(parent_path, folder_saved_models, file_name)
        df = load_data_one_stock(path)
        ticker_stats[ticker] = {
            'profit': -999,
            'wins': 0,
            'loses' : 0,
        }
        if df.shape[0] > 0:
          # profit, wins, loses= one_stock_simulation(df, ticker, mode='full')
          profit, wins, loses= one_stock_simulation(df, ticker,
                                                     mode='train',
                                                     _gain_coef=1.005,
                                                     _lose_coef=0.97)
          total_profit += profit
          total_loses += loses
          total_wins += wins
          ticker_stats[ticker] = {
            'profit': profit,
            'wins': wins,
            'loses' : loses,
          }
        else:
          stock_name_list_opt.remove(ticker)
      
      total_profit_list.append(total_profit)
      total_loses_list.append(total_loses)
      total_proportion_list.append(total_wins / (total_loses + 1))
      c.print(f'Total Profit is {total_profit:.2f}, total wins is {total_wins}, total loses is {total_loses} \
      proportion {total_wins/(total_loses + 1)}', color = 'red')
      # break if parametr is higher than previous value but profit decreased
      # if len(total_profit_list) > 1:
      #   if total_profit_list[-1] < total_profit_list[-2]:s
      #     break

    c.print(f'total_profit_list: {total_profit_list}', color='orange')
    if optimize_profit:
      vars()[parameter] = parameters_list_search[parameter][np.argmax(total_profit_list)]
    elif optimize_proportion:
      vars()[parameter] = parameters_list_search[parameter][np.argmax(total_proportion_list)]
    else:
      vars()[parameter] = parameters_list_search[parameter][np.argmin(total_loses_list)]
    print(f'parameter {parameter} best value is {vars()[parameter]}')
    # np.max(total_profit_list), np.argmax(total_profit_list)

  parameters_list_best = {}
  for parameter in parameters_list_search:
    print(f'parameter {parameter} best value is {vars()[parameter]}')
    
    parameters_list_best[parameter] = vars()[parameter]
    
  print(parameters_list_best)
  print(max(total_profit_list), min(total_loses_list))

  if optimize_proportion:
    best_profit_improved = max(total_proportion_list) > best_proportion
    best_proportion = max(total_proportion_list) if best_profit_improved else best_proportion
  else:
    best_profit_improved = max(total_profit_list) > best_profit
    best_profit = max(total_profit_list) if best_profit_improved else best_profit

  # Optimization which stocks to use:
  # for ticker in stock_name_list_opt:
  #   if ticker_stats[ticker]['wins'] / (ticker_stats[ticker]['loses'] + 1) < 3 \
  #       or ticker_stats[ticker]['profit'] < 0:
  #     stock_name_list_opt.remove(ticker)
  # c.print(f'Current optimal stock list len is {len(stock_name_list_opt)}, list is {stock_name_list_opt}', color='green')


parametr is RIV value is 0.05:
[Errno 2] No such file or directory: 'f:\\Git Projects\\HeikenAshiInvestingModel\\historical_data\\period2yinterval1h\\df_stock_PXD_period2y_interval1h.pkl'
Total Profit is -2468.41, total wins is 3460, total loses is 411       proportion 8.398058252427184
parametr is RIV value is 0.1:
Total Profit is -1434.60, total wins is 2782, total loses is 314       proportion 8.831746031746032
parametr is RIV value is 0.15:
Total Profit is -702.76, total wins is 2213, total loses is 234       proportion 9.417021276595746
parametr is RIV value is 0.2:
Total Profit is -91.07, total wins is 1726, total loses is 167       proportion 10.273809523809524
parametr is RIV value is 0.25:
Total Profit is 275.70, total wins is 1380, total loses is 122       proportion 11.21951219512195
total_profit_list: [np.float64(-2468.4069613184492), np.float64(-1434.6046056736639), np.float64(-702.7630749993673), np.float64(-91.07468894651035), np.float64(275.6954749093065)]
parameter RIV

KeyboardInterrupt: 

In [ ]:
print(stock_name_list_opt)
# len(stock_name_list_opt)

['GOOG', 'JPM', 'XOM', 'UNH', 'JNJ', 'V', 'AVGO', 'PG', 'LLY', 'MA', 'HD', 'CVX', 'MRK', 'PEP', 'COST', 'ABBV', 'ADBE', 'KO', 'CRM', 'WMT', 'MCD', 'CSCO', 'BAC', 'PFE', 'TMO', 'ACN', 'NFLX', 'ABT', 'AMD', 'LIN', 'ORCL', 'CMCSA', 'TXN', 'DIS', 'WFC', 'DHR', 'PM', 'NEE', 'VZ', 'INTC', 'RTX', 'HON', 'LOW', 'UPS', 'INTU', 'SPGI', 'NKE', 'COP', 'QCOM', 'BMY', 'CAT', 'UNP', 'BA', 'ISRG', 'GE', 'IBM', 'AMGN', 'AMAT', 'MDT', 'SBUX', 'PLD', 'NOW', 'MS', 'DE', 'BLK', 'GS', 'T', 'LMT', 'AXP', 'BKNG', 'SYK', 'ADI', 'TJX', 'ELV', 'MDLZ', 'GILD', 'ADP', 'MMC', 'C', 'AMT', 'CVS', 'VRTX', 'SCHW', 'LRCX', 'MO', 'TMUS', 'SLB', 'ETN', 'ZTS', 'CI', 'PYPL', 'FI', 'CB', 'SO', 'REGN', 'BSX', 'EQIX', 'BDX', 'PANW', 'DUK', 'EOG', 'MU', 'AON', 'ITW', 'CSX', 'SNPS', 'PGR', 'APD', 'KLAC', 'CME', 'NOC', 'CDNS', 'ICE', 'CL', 'SHW', 'WM', 'HCA', 'TGT', 'FCX', 'FDX', 'F', 'ORLY', 'MMM', 'CMG', 'EW', 'GM', 'MCK', 'NXPI', 'NSC', 'HUM', 'EMR', 'DXCM', 'PNC', 'PH', 'MPC', 'APH', 'ROP', 'FTNT', 'MCHP', 'USB', 'CCI', 'MAR'

188

# SOME RESULTS, HISTORY LOG

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1.001, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
6056.55094263172 1011
Current optimal stock list len is 127

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
7049.139814422607 599
Current optimal stock list len is 99,

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 10, 'bull_trend_coef': 1.12}
8616.535972952845 423
Current optimal stock list len is 97

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
8455.558336365704 662
Current optimal stock list len is 71

v1:
{'is_near_global_max_prt': 80, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.25, 'buy_ratio_border': 9, 'bull_trend_coef': 1.12}
5762.099450752256 170 , number_red_candles= 6

24/08/2024
{'is_near_global_max_prt': 100, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.25, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
119412.97672809978 380

In [100]:
for parameter in parameters_list_search:
  print(f'parameter {parameter} best value is {vars()[parameter]}')

  # if df['ha_pct'].iloc[i] > RIV \
  # and reverse_flag==False \
  # and sell_finished == True \
  # and last_top / df['open'].iloc[i] > last_top_ratio \
  # and i - last_top_i > distance_from_last_top \
  # and buy_time_cond \
  # and buy_ratio > buy_ratio_border \
  # and not(is_near_global_max(i, k=400, prt=is_near_global_max_prt)):

# Current_optimal_stock_list 
# stock_name_list_opt = ['JPM', 'UNH', 'AVGO', 'HD', 'PEP', 'COST', 'CRM', 'WMT',
#                                 'MCD', 'ABT', 'TXN', 'WFC', 'INTC', 'RTX', 'HON', 'SPGI',
#                                   'BA', 'AMGN', 'AMAT', 'SBUX', 'PLD', 'MS', 'DE', 'AXP', 
#                                   'ADI', 'TJX', 'MMC', 'CVS', 'SCHW', 'TMUS', 'ETN', 'CI', 
#                                   'BSX', 'AON', 'ITW', 'SNPS', 'KLAC', 'CDNS', 'WM', 'HCA',
#                                     'FDX', 'MMM', 'CMG', 'GM', 'MCK', 'NXPI', 'NSC',
#                                       'HUM', 'DXCM', 'PH', 'MPC', 'ROP', 'MCHP', 'MSI', 'GD',
#                                         'JCI', 'SRE', 'ADSK', 'AZO', 'TDG', 'AJG', 'TT', 'PCAR',
#                                           'COF', 'CTAS', 'AIG', 'ANET', 'SPG', 'MSCI', 'ROK']

stock_name_list_opt = [
'BA', 'ON', 'MCHP', 'ADI', 'PANW', 'DHI', 'ANET', 'AMD', 'LRCX', 'LLY',
'MU', 'TXN', 'AIG', 'WMB', 'BSX', 'NKE', 'OXY', 'TT', 'AMAT', 'ETN', 'DE',
'EL', 'FDX', 'MAR', 'GE', 'NFLX', 'NUE', 'GOOG', 'ECL', 'AVGO', 'CAT', 'SPG',
'ADSK', 'INTU', 'SLB', 'F', 'WMT', 'SBUX', 'SNPS', 'AJG', 'TMUS', 'KLAC', 'CI',
'JCI', 'GILD', 'QCOM', 'ROP', 'MO', 'WM', 'HON', 'ITW', 'GS', 'HCA', 'TJX', 'ICE',
'DXCM', 'IDXX', 'ABBV', 'CDNS', 'CMCSA', 'JNJ', 'EQIX', 'MDLZ', 'NXPI', 'MSI', 'TEL',
'LMT', 'USB', 'MRK', 'HLT', 'APD', 'CTAS', 'MNST', 'NOW', 'AMT', 'PH', 'HUM', 'ADM',
'TDG', 'EMR', 'GM', 'ADP', 'CMG', 'SCHW', 'MSCI', 'EOG', 'UNP', 'INTC', 'CME',
'MA', 'CVS', 'XOM', 'CSCO', 'WELL', 'TMO', 'MRNA', 'PLD', 'APH', 'PEP', 'CRM', 'MMM',
'MMC', 'LIN', 'GIS', 'COST', 'CSX', 'IQV', 'FI', 'MCD', 'VRTX'
]

# stock_name_list_opt =['GOOG', 'JPM', 'XOM', 'UNH', 'JNJ', 'V', 'AVGO', 'PG', 'LLY', 'MA', 'HD', 'CVX', 'MRK', 'PEP', 'COST', 'ABBV', 'ADBE', 'KO', 'CRM', 'WMT', 'MCD', 'CSCO', 'BAC', 'PFE', 'TMO', 'ACN', 'NFLX', 'ABT', 'AMD', 'LIN', 'ORCL', 'CMCSA', 'TXN', 'DIS', 'WFC', 'DHR', 'PM', 'NEE', 'VZ', 'INTC', 'RTX', 'HON', 'LOW', 'UPS', 'INTU', 'SPGI', 'NKE', 'COP', 'QCOM', 'BMY', 'CAT', 'UNP', 'BA', 'ISRG', 'GE', 'IBM', 'AMGN', 'AMAT', 'MDT', 'SBUX', 'PLD', 'NOW', 'MS', 'DE', 'BLK', 'GS', 'T', 'LMT', 'AXP', 'BKNG', 'SYK', 'ADI', 'TJX', 'ELV', 'MDLZ', 'GILD', 'ADP', 'MMC', 'C', 'AMT', 'CVS', 'VRTX', 'SCHW', 'LRCX', 'MO', 'TMUS', 'SLB', 'ETN', 'ZTS', 'CI', 'PYPL', 'FI', 'CB', 'SO', 'REGN', 'BSX', 'EQIX', 'BDX', 'PANW', 'DUK', 'EOG', 'MU', 'AON', 'ITW', 'CSX', 'SNPS', 'PGR', 'APD', 'KLAC', 'CME', 'NOC', 'CDNS', 'ICE', 'CL', 'SHW', 'WM', 'HCA', 'TGT', 'FCX', 'FDX', 'F', 'ORLY', 'MMM', 'CMG', 'EW', 'GM', 'MCK', 'NXPI', 'NSC', 'HUM', 'EMR', 'DXCM', 'PNC', 'PH', 'MPC', 'APH', 'ROP', 'FTNT', 'MCHP', 'USB', 'CCI', 'MAR', 'MSI', 'GD', 'PSA', 'JCI', 'PSX', 'SRE', 'ADSK', 'AZO', 'TDG', 'ECL', 'AJG', 'KMB', 'TEL', 'TT', 'AEP', 'EL', 'PCAR', 'OXY', 'TFC', 'CARR', 'D', 'IDXX', 'GIS', 'ON', 'COF', 'ADM', 'MNST', 'NUE', 'CTAS', 'AIG', 'EXC', 'VLO', 'MRNA', 'ANET', 'WMB', 'O', 'STZ', 'IQV', 'HLT', 'CHTR', 'WELL', 'BIIB', 'SPG', 'MSCI', 'DHI', 'ROK']


len(stock_name_list_opt), len(stock_name_list)


parameter is_near_global_max_prt best value is 96
parameter distance_from_last_top best value is 0
parameter last_top_ratio best value is 1.01
parameter RIV best value is 0.05
parameter buy_ratio_border best value is 0
parameter bull_trend_coef best value is 1.07


(110, 188)

In [101]:
len(stock_name_list), len(stock_name_list_opt)

(188, 110)

In [104]:
# TRY TO REDUCE NUMBER OF FAIL NOT PROFIT!!!
total_profit_list = []
# parametr_list = np.arange(86, 100, 1)
# parametr_list = [1, 1.001, 1.002, 1.005, 1.007, 1.01, 1.02]
# parametr_list = [30]
# for p in parametr_list:
#   is_near_global_max_prt= p
 
# print("TRAIN RESULTS")
# total_profit = 0
# total_loses = 0
# total_wins = 0
# for ticker in stock_name_list_opt:
#   file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
#   path = os.path.join(parent_path, folder_saved_models, file_name)
#   df = load_data_one_stock(path)
#   profit, wins, loses = one_stock_simulation(df, ticker, mode = 'train', print_sim_results=True, print_trans_history=False)
#   total_profit += profit
#   total_loses += loses
#   total_wins += wins
# print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_wins / total_loses + 1:.2f}')

print("TEST RESULTS")
total_profit = 0
total_loses = 0
total_wins = 0
for ticker in stock_name_list_opt:
# for ticker in stock_name_list:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)
  if df.shape[0] > 0:
    profit, wins, loses = one_stock_simulation(df, ticker, mode = 'test', print_sim_results=True, print_trans_history=False)
    total_profit += profit
    total_loses += loses
    total_wins += wins
    print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_wins / (total_loses + 1):.2f}')
  else:
    print(f'{ticker} empty df')


TEST RESULTS
--------------------------------------------------
Simulation results for ticker JPM:
{'number success': 26, 'number fails': 3, 'number wins': 26, 'proportion': 6.5, 'profit': 55.73440264892578, 'transactions': [{'type': 'fail', 'buy_price': 150.83, 'sell_price': 142.65, 'ratio': 10.85, 'buy_time': Timestamp('2023-08-17 09:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-09-08 09:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': 144.71, 'sell_price': 145.62, 'ratio': 1.16, 'buy_time': Timestamp('2023-09-11 09:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-09-12 09:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': 145.43, 'sell_price': 146.52, 'ratio': 15.11, 'buy_time': Timestamp('2023-09-25 12:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-09-25 15:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': 145.86, 'sell_price': 147.94, 'ratio': 3.87, 'buy_time': Ti

In [25]:
# best_params = {'is_near_global_max_prt': 96, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.05, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07}
# best_params = {'is_near_global_max_prt': 30, 'distance_from_last_top': 0, 'last_top_ratio': 1.1, 'RIV': 0.1, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07}
# best_params = {'is_near_global_max_prt': 20, 'distance_from_last_top': 0, 'last_top_ratio': 1.1, 'RIV': 0.05, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07}
# best_params = {'is_near_global_max_prt': 70, 'distance_from_last_top': 0, 'last_top_ratio': 1.1, 'RIV': 0.2, 'buy_ratio_border': 7, 'bull_trend_coef': 1.07}
best_params = {'is_near_global_max_prt': 120, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07,
              'is_near_global_max_prt2': 60,  'global_max_prop_ratio' : 1.08}
for param in best_params:
  vars()[param] = best_params[param]

stock_name_list_opt = ['MAR', 'MO', 'DHI', 'PYPL', 'HLT', 'BIIB', 'ZTS', 'TMUS', 'CMG', 'WELL', 'DUK',
                       'MCHP', 'MS', 'ROK', 'INTU', 'EXC', 'CI', 'STZ', 'NXPI', 'COF', 'DIS', 'EL', 'MU',
                       'NOW', 'CAT', 'HUM', 'OXY', 'PSX', 'COP', 'QCOM', 'ON', 'CVS', 'VLO', 'TJX', 'PGR',
                       'TXN', 'EOG', 'CSX', 'PM', 'MMM', 'ADM', 'GM', 'PEP', 'VZ', 'REGN', 'PCAR', 'GILD',
                       'EW', 'ANET', 'GIS', 'LIN', 'TEL', 'VRTX', 'BLK', 'ELV', 'KO', 'DE', 'KLAC', 'SBUX',
                       'PLD', 'PANW', 'CHTR', 'SNPS', 'MCK', 'TFC', 'USB', 'TDG', 'PFE', 'MRK', 'CB', 'MPC',
                       'GOOG', 'TT', 'PSA', 'CME', 'AIG', 'IDXX', 'BSX', 'WFC', 'ABT', 'TGT', 'EQIX', 'CTAS',
                       'BA', 'AON', 'SYK', 'F', 'SRE', 'JCI', 'NKE', 'MCD', 'TMO', 'SO', 'WMT', 'UNH', 'CSCO',
                      'CRM', 'PG', 'JNJ', 'HCA', 'XOM', 'ETN', 'V', 'COST', 'GS', 'FCX', 'MDT', 'NEE', 'CL',
                       'BMY', 'SPG', 'ADI', 'ABBV', 'WM', 'NUE', 'MDLZ', 'MA', 'MRNA', 'ROP', 'AEP', 'IBM',
                      'EMR', 'JPM', 'APD', 'AMD', 'T', 'INTC', 'ITW', 'AMGN', 'ADSK', 'SHW', 'NOC', 'NSC', 'SCHW', 'ICE']

In [42]:
print("FULL RESULTS OPTIMIZATION")
# parameters:
mode = 'full'
lose_coef = 0.97
gain_coef = 1.005
# gain_coef = 1.02
use_optimal_stock_list = False
use_stock_name_list_selected = True
#
total_profit = 0
total_loses = 0
total_wins = 0

if use_optimal_stock_list:
  current_stock_name_list = stock_name_list_opt[:]
else:
  current_stock_name_list = stock_name_list[:]

if use_stock_name_list_selected:
  current_stock_name_list = stock_name_list_selected[:]

# current_stock_name_list = ['CME', 'AMT']

df_opt = pd.DataFrame(columns=['ticker','profit', 'wins', 'loses', 'proportion'])

for ticker in current_stock_name_list:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)

  # if ticker in opt_stocks_for_bear_trend:
  #   gain_coef = 1.02
  # else:
  #   gain_coef = 1.005

  if df.shape[0] > 0:
    profit, wins, loses = one_stock_simulation(df, ticker,
                                               mode=mode,
                                               _gain_coef=gain_coef,
                                               _lose_coef=lose_coef, 
                                               print_sim_results=True, 
                                               print_trans_history=True)
    if loses == 0:
      proportion = wins
    else:
      proportion = wins / loses
    df2 = pd.DataFrame({'ticker': [ticker],'profit': [profit], 'wins': [wins], 'loses': [loses], 'proportion': [proportion]})
    df_opt = pd.concat([df_opt, df2], ignore_index=True)
    total_profit += profit
    total_loses += loses
    total_wins += wins
    if total_loses == 0:
      total_proportion = total_wins
    else:
      total_proportion = total_wins / total_loses
  else:
    print(f'{ticker} empty df')
c.print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_proportion:.2f}', color = 'blue')
c.print('Parameters are:', color = 'blue' )
c.print(f'gain_coef is {gain_coef}; lose_coef is {lose_coef}; use_optimal_stock_list is {use_optimal_stock_list}, use_selected_optimal_stock_list is \
{use_stock_name_list_selected} mode is {mode}', color = 'blue')
for param in best_params:
   print(f'{param} is {vars()[param]}')

FULL RESULTS OPTIMIZATION
--------------------------------------------------
Simulation results for ticker DXCM:
{'number success': 34, 'number fails': 0, 'number wins': 34, 'proportion': 34.0, 'profit': np.float64(95.55836751758227), 'transactions': [{'type': 'success', 'buy_price': np.float64(88.56), 'sell_price': np.float64(89.0), 'ratio': 2.92, 'buy_time': Timestamp('2022-09-16 15:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-09-19 09:30:00-0400', tz='America/New_York'), 'buy_lose_coef': 0.97, 'buy_gain_coef': 1.005}, {'type': 'success', 'buy_price': np.float64(85.82), 'sell_price': np.float64(86.25), 'ratio': 2.91, 'buy_time': Timestamp('2022-09-19 15:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-09-20 09:30:00-0400', tz='America/New_York'), 'buy_lose_coef': 0.97, 'buy_gain_coef': 1.005}, {'type': 'success', 'buy_price': np.float64(80.0), 'sell_price': np.float64(80.4), 'ratio': 9.11, 'buy_time': Timestamp('2022-09-22 15:30:00-0400', tz='Ame

C:\Users\IvanSpok\AppData\Local\Temp\ipykernel_18264\2231791883.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_opt = pd.concat([df_opt, df2], ignore_index=True)


--------------------------------------------------
Simulation results for ticker BIIB:
{'number success': 31, 'number fails': 2, 'number wins': 31, 'proportion': 10.333333333333334, 'profit': np.float64(20.70061462012966), 'transactions': [{'type': 'success', 'buy_price': np.float64(203.45), 'sell_price': np.float64(204.47), 'ratio': 8.01, 'buy_time': Timestamp('2022-09-21 12:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-09-21 13:30:00-0400', tz='America/New_York'), 'buy_lose_coef': 0.97, 'buy_gain_coef': 1.005}, {'type': 'success', 'buy_price': np.float64(262.91), 'sell_price': np.float64(264.22), 'ratio': 44.22, 'buy_time': Timestamp('2022-10-20 09:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-10-20 10:30:00-0400', tz='America/New_York'), 'buy_lose_coef': 0.97, 'buy_gain_coef': 1.005}, {'type': 'success', 'buy_price': np.float64(291.05), 'sell_price': np.float64(292.51), 'ratio': 42.82, 'buy_time': Timestamp('2022-11-29 11:30:00-0500', tz='Amer

Results:
1. use ordered selected stock list;
2. Use danamic gain coeficient'


In [28]:
df_opt = df_opt.sort_values(by='profit', ascending=False)
print(df_opt)
df_opt.to_csv('df_opt.csv')

    ticker      profit wins loses  proportion
130   DXCM   64.822255   23     0   23.000000
182   BIIB   59.249269   23     0   23.000000
139    CCI   55.064477   20     0   20.000000
79     CVS   54.655403   19     0   19.000000
34     WFC   47.308633   16     0   16.000000
..     ...         ...  ...   ...         ...
22     BAC  -77.404055   16     4    4.000000
77       C  -78.751705   16     4    4.000000
172    VLO -107.287457    5     4    1.250000
46     NKE -125.436323   10     5    2.000000
131    PNC -171.000497   12     7    1.714286

[187 rows x 5 columns]


In [29]:
stock_name_list_selected = df_opt.loc[df_opt['profit'] > 0].sort_values(by='profit', ascending=False)['ticker']
print(stock_name_list_selected.to_list())
print(len(stock_name_list_selected))
# stock_name_for_check = [
# 'BA', 'ON', 'MCHP', 'ADI', 'PANW', 'DHI', 'ANET', 'AMD', 'LRCX', 'LLY',
# 'MU', 'TXN', 'AIG', 'WMB', 'BSX', 'NKE', 'OXY', 'TT', 'AMAT', 'ETN', 'DE',
# 'EL', 'FDX', 'MAR', 'GE', 'NFLX', 'NUE', 'GOOG', 'ECL', 'AVGO', 'CAT', 'SPG',
# 'ADSK', 'INTU', 'SLB', 'F', 'WMT', 'SBUX', 'SNPS', 'AJG', 'TMUS', 'KLAC', 'CI',
# 'JCI', 'GILD', 'QCOM', 'ROP', 'MO', 'WM', 'HON', 'ITW', 'GS', 'HCA', 'TJX', 'ICE',
# 'DXCM', 'IDXX', 'ABBV', 'CDNS', 'CMCSA', 'JNJ', 'EQIX', 'MDLZ', 'NXPI', 'MSI', 'TEL',
# 'LMT', 'USB', 'MRK', 'HLT', 'APD', 'CTAS', 'MNST', 'NOW', 'AMT', 'PH', 'HUM', 'ADM',
# 'TDG', 'EMR', 'GM', 'ADP', 'CMG', 'SCHW', 'MSCI', 'EOG', 'UNP', 'INTC', 'CME',
# 'MA', 'CVS', 'XOM', 'CSCO', 'WELL', 'TMO', 'MRNA', 'PLD', 'APH', 'PEP', 'CRM', 'MMM',
# 'MMC', 'LIN', 'GIS', 'COST', 'CSX', 'IQV', 'FI', 'MCD', 'VRTX'
# ]
# (stock_name_for_check == stock_name_list_selected).all()
opt_stocks_for_bear_trend = ['BA', 'INTU', 'MCHP', 'LLY', 'DHI', 'ANET', 'AIG', 'NUE', 'MAR', 'OXY', 'ON',
  'GE', 'AMAT', 'NXPI', 'SNPS', 'UNP', 'KLAC', 'BSX', 'MSI', 'CRM', 'CAT', 'ADI',
    'ETN', 'JCI', 'HLT', 'CSCO', 'WMT', 'TDG', 'TT', 'ECL', 'LOW', 'ADSK', 'TJX',
    'VRTX', 'APH', 'ABBV', 'STZ', 'SBUX', 'DE', 'MRK', 'CTAS', 'MNST', 'CME', 'MO', 'TXN', 'ITW']
# print(stock_name_list_selected)
# df_opt.loc[df_opt['profit'] > 0].sort_values(by='profit', ascending=False)


['DXCM', 'BIIB', 'CCI', 'CVS', 'WFC', 'CSX', 'TXN', 'HD', 'DUK', 'ABBV', 'KO', 'PSA', 'BA', 'PYPL', 'SBUX', 'ADM', 'FDX', 'PGR', 'ADP', 'JCI', 'TEL', 'WM', 'IDXX', 'NUE', 'ITW', 'PCAR', 'SPG', 'BDX', 'PEP', 'KMB', 'ROK', 'DE', 'AMGN', 'MNST', 'SNPS', 'CL', 'MMC', 'BLK', 'AJG', 'MDLZ', 'AMD', 'CB', 'SCHW', 'AVGO', 'WELL', 'MSCI', 'MRNA', 'OXY', 'ADSK', 'APH', 'IBM', 'CHTR', 'EL', 'MDT', 'GS', 'NOC', 'ELV', 'USB', 'AEP', 'SRE', 'LMT', 'LIN', 'WMT', 'TT', 'PSX', 'UNP', 'MCD', 'CMG', 'ROP', 'QCOM', 'CVX', 'SO', 'NEE', 'UNH', 'CDNS', 'AXP', 'MRK', 'CMCSA', 'MO', 'ORCL', 'NFLX', 'MPC', 'FTNT', 'MU', 'ECL', 'JPM', 'SHW', 'CSCO', 'ETN', 'GOOG', 'SYK', 'MCHP']
92


In [120]:
df_opt_bear_trend = df_opt.sort_values(by='proportion', ascending=False)

df_opt_bear_trend = df_opt_bear_trend.loc[df_opt_bear_trend['proportion'] >= 3].sort_values(by='profit', ascending=False)
print(df_opt_bear_trend)
print(df_opt_bear_trend['ticker'].to_list())

    ticker      profit wins loses  proportion
0       BA  674.602803   38     6    6.333333
8     INTU  657.527402   40     7    5.714286
6     MCHP  599.032862   41     8    5.125000
13     LLY  587.327168   26     4    6.500000
2      DHI  573.843221   36     8    4.500000
17    ANET  563.029703   36    12    3.000000
11     AIG  491.698669   32     7    4.571429
24     NUE  465.124099   39    12    3.250000
31     MAR  456.696729   33     6    5.500000
1      OXY  455.211293   36    10    3.600000
3       ON  444.715145   45    13    3.461538
21      GE  440.864404   28     5    5.600000
15    AMAT  410.183924   36    11    3.272727
10    NXPI  404.862433   38    10    3.800000
28    SNPS  376.630274   33    10    3.300000
87     UNP  360.880210   23     6    3.833333
30    KLAC  358.535270   32     8    4.000000
7      BSX  349.135627   20     4    5.000000
61     MSI  348.189323   26     5    5.200000
85     CRM  330.056228   33    11    3.000000
43     CAT  329.017333   26     7 